In [1]:
import librosa as lb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
def calculate_bpm(audio_path):
    y, sr = lb.load(audio_path)
    onset_env = lb.onset.onset_strength(y=y, sr=sr)
    tempo, _ = lb.beat.beat_track(onset_envelope=onset_env, sr=sr)
    return tempo

bpm = calculate_bpm('./goat.mp3')
bpm

array([107.66601562])

In [ ]:
def calc_speed(bpm, changed_bpm):
    return changed_bpm/bpm

In [ ]:
def change_speed_and_pitch(audio_path, speed_factor, output_path):
    
    # Change speed (tempo) but keep pitch
    y_fast = lb.effects.time_stretch(y, rate=speed_factor)
    
    sf.write(output_path, y_fast, sr)
    print(f"Processed file saved at {output_path}")

# Example Usage
changed_bpm = bpm * 1.5
change_speed_and_pitch("input.mp3", calc_speed(bpm, changed_bpm), "output_fixed.mp3")  # 1.2x speed

In [ ]:
def change_speed_and_pitch(audio_data, sr, speed_factor):
    """
    Changes the speed of audio data without saving to file
    Parameters:
        audio_data: numpy array of audio samples
        sr: sample rate
        speed_factor: factor to change speed (e.g., 1.2 for 20% faster)
    Returns:
        processed audio data as numpy array
    """
    # Change speed (tempo) but keep pitch
    y_processed = lb.effects.time_stretch(audio_data, rate=speed_factor)
    
    return y_processed, sr

In [ ]:
# Load audio once
y, sr = lb.load("input.mp3")

# Process audio when speed change is requested (e.g., from a slider in UI)
def on_speed_change(speed_factor):
    processed_audio, sr = change_speed_and_pitch(y, sr, speed_factor)
    # Here you would send processed_audio to your audio playback system
    return processed_audio